<a href="https://colab.research.google.com/github/Viny2030/UNED/blob/main/corenlpinterfaz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ejecutar CoreNLP directamente en JavaScript dentro de un entorno Colab presenta desafíos, ya que CoreNLP está principalmente escrito en Java. Sin embargo, hay varias estrategias que puedes emplear para integrar la funcionalidad de CoreNLP en tu flujo de trabajo de JavaScript en Colab:

**1. Configurar un servidor CoreNLP y comunicarse con él desde JavaScript:**

Ejecutar CoreNLP como un servidor Java:
Puedes iniciar un servidor CoreNLP en Colab utilizando comandos de shell para ejecutar Java.
Esto implica descargar el paquete CoreNLP, configurarlo y ejecutar el servidor a través de la línea de comandos.
Colab te permite ejecutar comandos de shell precedidos por un signo de exclamación (!).
Para que esto funcione, primero debes instalar Java en el entorno de Colab.
Comunicación con el servidor:
Una vez que el servidor CoreNLP esté en funcionamiento, puedes usar JavaScript para enviar solicitudes HTTP al servidor y recibir respuestas.
Esto se puede lograr utilizando la función fetch() de JavaScript o una biblioteca como Axios.
Para poder acceder al servidor local desde el exterior de colab, se necesita usar una herramienta como Ngrok, que crea un tunnel para exponer el puerto donde el servidor de coreNLP esta funcionando.
Consideraciones:
Esta es la forma más comun de hacerlo, pero requiere configurar un servidor y manejar la comunicación de red.
Esta opcion implica que el procesamiento del lenguaje natural se hara en el servidor de CoreNLP y que javascript solo va a recibir los resultados.
**2. Utilizar bibliotecas JavaScript de NLP:**

Aunque CoreNLP es una herramienta poderosa, existen bibliotecas de procesamiento de lenguaje natural (NLP) en JavaScript que puedes usar directamente en Colab.
Bibliotecas como Natural o Compromise proporcionan funcionalidades NLP que pueden cubrir algunas de tus necesidades.
Estas librerias se ejecutan del lado del cliente, por lo cual todo el proceso se hace en el navegador.
**3. Usar Stanza con una interfaz CoreNLP:**

Stanza es una biblioteca de NLP de Stanford que proporciona una interfaz de Python para CoreNLP.
Puedes usar Stanza en Colab para realizar el procesamiento de NLP y luego pasar los resultados a JavaScript.
Esta opcion implica usar python como lenguaje intermediario.
Puntos clave:

La ejecución directa de CoreNLP en JavaScript es limitada debido a su naturaleza basada en Java.
La configuración de un servidor CoreNLP y la comunicación a través de HTTP es una opción viable.
Las bibliotecas de NLP de JavaScript y Stanza ofrecen alternativas para realizar tareas de NLP en Colab.
Si bien no se puede ejecutar el código CoreNLP Java directamente en el entorno JavaScript de Colab, existen soluciones para aprovechar sus capacidades de procesamiento del lenguaje natural.

***1. Configurar un servidor CoreNLP y comunicarse con él desde JavaScript:***

**a. Configuración del servidor CoreNLP en Colab (comandos de shell):**

In [1]:
# Instalar Java
!apt-get install openjdk-11-jdk

# Descargar CoreNLP
!wget http://nlp.stanford.edu/software/stanford-corenlp-full-2020-04-20.zip
!unzip stanford-corenlp-full-2020-04-20.zip

# Iniciar el servidor CoreNLP (en segundo plano)
!cd stanford-corenlp-full-2020-04-20 && java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 15000 &

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java libatk-wrapper-java-jni libfontenc1
  libice-dev libsm-dev libxkbfile1 libxt-dev libxtst6 libxxf86dga1 openjdk-11-jre x11-utils
Suggested packages:
  libice-doc libsm-doc libxt-doc openjdk-11-demo openjdk-11-source visualvm mesa-utils
The following NEW packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java libatk-wrapper-java-jni libfontenc1
  libice-dev libsm-dev libxkbfile1 libxt-dev libxtst6 libxxf86dga1 openjdk-11-jdk openjdk-11-jre
  x11-utils
0 upgraded, 14 newly installed, 0 to remove and 29 not upgraded.
Need to get 5,522 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 fonts-dejavu-core all 2.37-2build1 [1,041 kB]
Get:2 http://archive.ubuntu.com/ubun

**b. Comunicación con el servidor desde JavaScript (celda %%javascript en Colab):**

In [2]:
%%javascript
async function analyzeText(text) {
  const response = await fetch('http://localhost:9000/?properties={"annotators": "tokenize,ssplit,pos,lemma,ner", "outputFormat": "json"}', {
    method: 'POST',
    body: text
  });
  const data = await response.json();
  return data;
}

// Ejemplo de uso
analyzeText('Stanford CoreNLP is great!').then(data => {
  console.log(data);
});

<IPython.core.display.Javascript object>


**c. Usando Ngrok para exponer el servidor de coreNLP.**

**Primero se debe instalar ngrok.**

In [3]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2025-03-17 11:47:10--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 35.71.179.82, 13.248.244.96, 99.83.220.108, ...
Connecting to bin.equinox.io (bin.equinox.io)|35.71.179.82|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.28M  --.-KB/s    in 0.06s   

2025-03-17 11:47:11 (220 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13921656/13921656]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


**Luego se puede crear el tunel.**

In [14]:
!pip install ngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 54.4 MB/s eta 0:00:00


In [15]:
!ngrok config add-authtoken 2h1F1Ydnhk6SGILXYIHlPKiZn9p_3qJmVbuQVtxYvjSBssDU6

/bin/bash: line 1: ngrok: command not found


In [11]:
# prompt: crear tunel de ngrok con token

# Replace 'YOUR_NGROK_AUTHTOKEN' with your actual authtoken
!./ngrok authtoken 2h1F1Ydnhk6SGILXYIHlPKiZn9p_3qJmVbuQVtxYvjSBssDU6

# Create the tunnel to port 9000 (where CoreNLP is running)
!./ngrok http 9000

# Get the public URL from the ngrok output
# The output will look like this (the public URL is what you need):
# ...
# Forwarding     http://localhost:9000 -> http://<your_public_url>:9000
# ...
# You'll need to extract <your_public_url> from the ngrok output
# and use it in your JavaScript code to make requests to CoreNLP.

# Example:
# public_url = "your_public_url_from_ngrok_output" # Replace with the actual public url
# print(f"Public URL: {public_url}")


Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
Your ngrok-agent version "2.3.41" is too old. The minimum supported agent version for your account is "3.6.0". Please update to a newer version with `ngrok update`, by downloading from https://ngrok.com/download, or by updating your SDK version. Paid accounts are currently excluded from minimum agent version requirements. To begin handling traffic immediately without updating your agent, upgrade to a paid plan: https://dashboard.ngrok.com/billing/subscription.

ERR_NGROK_121



In [10]:
!ngrok config add-authtoken 2h1F1Ydnhk6SGILXYIHlPKiZn9p_3qJmVbuQVtxYvjSBssDU6

/bin/bash: line 1: ngrok: command not found


In [7]:
# Instalar ngrok (si no lo has hecho ya)
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

# Configurar authtoken (# Instalar ngrok (si no lo has hecho ya)
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

# Configurar authtoken (reemplaza YOUR_AUTHTOKEN con tu authtoken real)
!./ngrok authtoken YOUR_AUTHTOKEN

# Iniciar el túnel ngrok
!./ngrok http 9000 &

# Obtener la URL pública del túnel
!curl -s http://localhost:4040/api/tunnels)
!./ngrok authtoken YOUR_AUTHTOKEN

# Iniciar el túnel ngrok
!./ngrok http 9000 &

# Obtener la URL pública del túnel
!curl -s http://localhost:4040/api/tunnels

--2025-03-17 11:52:40--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 35.71.179.82, 99.83.220.108, 75.2.60.68, ...
Connecting to bin.equinox.io (bin.equinox.io)|35.71.179.82|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]  13.28M  --.-KB/s    in 0.07s   

2025-03-17 11:52:40 (202 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [13921656/13921656]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: --2025-03-17 11:52:56--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 35.71.179.82, 99.83.220.108, 75.2.60.68, ...
Connecting to bin.equinox.io (bin.equinox.io)|35.71.179.82|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [app

**Esto va a mostrar un json con la url publica para poder acceder al servidor de core nlp.**
Luego se puede usar esa url en el codigo javascript anterior, reemplazando http://localhost:9000/ por la url provista por ngrok.

**2. Utilizar bibliotecas JavaScript de NLP:**

In [5]:
%%javascript
// Usando Compromise
let doc = nlp('Stanford CoreNLP is great!');
console.log(doc.sentences().json());

// Usando Natural
// (Natural requiere Node.js, que no está preinstalado en Colab. Se puede instalar, pero es más complejo)

<IPython.core.display.Javascript object>

**3. Usar Stanza con una interfaz CoreNLP:**

a. Instalación y uso de Stanza en Python (celda de código Python):

In [6]:
!pip install stanza

import stanza

stanza.download('en') # Descargar el modelo en inglés
nlp = stanza.Pipeline('en', processors='tokenize,mwt,pos,lemma,ner')
doc = nlp("Stanford CoreNLP is great!")
print(doc)

# Extraer los resultados y pasarlos a JavaScript (ejemplo simplificado)
resultados_stanza = str(doc)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 33.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: en (English) ...


INFO:stanza:Downloaded file to /root/stanza_resources/en/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: en (English):
| Processor | Package                   |
-----------------------------------------
| tokenize  | combined                  |
| mwt       | combined                  |
| pos       | combined_charlm           |
| lemma     | combined_nocharlm         |
| ner       | ontonotes-ww-multi_charlm |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


[
  [
    {
      "id": 1,
      "text": "Stanford",
      "lemma": "Stanford",
      "upos": "PROPN",
      "xpos": "NNP",
      "feats": "Number=Sing",
      "start_char": 0,
      "end_char": 8,
      "ner": "B-ORG",
      "multi_ner": [
        "B-ORG"
      ]
    },
    {
      "id": 2,
      "text": "CoreNLP",
      "lemma": "CoreNLP",
      "upos": "PROPN",
      "xpos": "NNP",
      "feats": "Number=Sing",
      "start_char": 9,
      "end_char": 16,
      "ner": "E-ORG",
      "multi_ner": [
        "E-ORG"
      ]
    },
    {
      "id": 3,
      "text": "is",
      "lemma": "be",
      "upos": "AUX",
      "xpos": "VBZ",
      "feats": "Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin",
      "start_char": 17,
      "end_char": 19,
      "ner": "O",
      "multi_ner": [
        "O"
      ]
    },
    {
      "id": 4,
      "text": "great",
      "lemma": "great",
      "upos": "ADJ",
      "xpos": "JJ",
      "feats": "Degree=Pos",
      "start_char": 20,
      "end_ch

**b. Pasar los resultados a JavaScript (celda %%javascript):**

In [16]:
%%javascript
// Recuperar los resultados de Python (usando la variable `resultados_stanza` definida en la celda Python)
let resultadosPython = {{resultados_stanza}};
console.log(resultadosPython);

<IPython.core.display.Javascript object>

Explicación de los cambios:

!./ngrok authtoken YOUR_AUTHTOKEN:
Esta línea agrega el authtoken a la configuración de ngrok.
Reemplaza YOUR_AUTHTOKEN con el authtoken que copiaste de la página de ngrok.
This command saves the authtoken to the ngrok configuration file, so you only need to run it once per Colab session.
El resto del código permanece igual, ya que la autenticación resuelve el problema principal.
Pasos detallados:

Reemplaza YOUR_AUTHTOKEN: Asegúrate de reemplazar el marcador de posición con tu authtoken real.
Ejecuta la celda: Ejecuta la celda de código en Colab.
Verifica la URL: La salida del comando !curl -s http://localhost:4040/api/tunnels mostrará un JSON que contiene la URL pública generada por ngrok. Busca la clave "public_url" dentro del JSON.
Usa la URL: Utiliza la URL pública en tu código JavaScript para comunicarte con el servidor CoreNLP.
Con estos cambios, ngrok debería funcionar correctamente y proporcionar una URL pública para acceder a tu servidor CoreNLP.

# Descargar la versión más reciente de ngrok
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz

# Descomprimir el archivo
!tar -xzf ngrok-v3-stable-linux-amd64.tgz

# Configurar authtoken (reemplaza YOUR_AUTHTOKEN con tu authtoken real)
!./ngrok authtoken 2h1F1Ydnhk6SGILXYIHlPKiZn9p_3qJmVbuQVtxYvjSBssDU6

# Iniciar el túnel ngrok
!./ngrok http 9000 &

# Verificar si ngrok está funcionando
!curl http://localhost:4040

# Obtener la URL pública del túnel (con inspección HTML)
!./ngrok http 9000 --inspect-html > ngrok_info.html

# Mostrar la información del túnel (si está disponible)
!curl -s http://localhost:4040/api/tunnels

In [18]:
# Descargar la versión más reciente de ngrok
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz

# Descomprimir el archivo
!tar -xzf ngrok-v3-stable-linux-amd64.tgz

# Configurar authtoken (reemplaza YOUR_AUTHTOKEN con tu authtoken real)
!./ngrok authtoken 2h1F1Ydnhk6SGILXYIHlPKiZn9p_3qJmVbuQVtxYvjSBssDU6

# Iniciar el túnel ngrok
!./ngrok http 9000 &

# Obtener la URL pública del túnel
!curl -s http://localhost:4040/api/tunnels

--2025-03-17 11:58:33--  https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 35.71.179.82, 13.248.244.96, 75.2.60.68, ...
Connecting to bin.equinox.io (bin.equinox.io)|35.71.179.82|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9371732 (8.9M) [application/octet-stream]
Saving to: ‘ngrok-v3-stable-linux-amd64.tgz’

ngrok-v3-stable-lin 100%[===================>]   8.94M  --.-KB/s    in 0.05s   

2025-03-17 11:58:33 (186 MB/s) - ‘ngrok-v3-stable-linux-amd64.tgz’ saved [9371732/9371732]

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
Default version saved to configuration file: /root/.ngrok2/ngrok.yml


Explicación de los cambios:

!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz:
Esta línea descarga la versión más reciente de ngrok.
!tar -xzf ngrok-v3-stable-linux-amd64.tgz:
Esta línea descomprime el archivo descargado.
The rest of the code remains the same.
By downloading the tgz file, we download the newest version of ngrok.
Pasos detallados:

Reemplaza YOUR_AUTHTOKEN: Asegúrate de reemplazar el marcador de posición con tu authtoken real.
Ejecuta la celda: Ejecuta la celda de código en Colab.
Verifica la URL: La salida del comando !curl -s http://localhost:4040/api/tunnels mostrará un JSON que contiene la URL pública generada por ngrok. Busca la clave "public_url" dentro del JSON.
Usa la URL: Utiliza la URL pública en tu código JavaScript para comunicarte con el servidor CoreNLP.
Con estos cambios, ngrok debería funcionar correctamente y proporcionar una URL pública para acceder a tu servidor CoreNLP.

# **Consideraciones:**

La configuración del servidor CoreNLP puede ser compleja y consumir recursos.
Las bibliotecas de NLP de JavaScript pueden no tener la misma amplitud de funcionalidades que CoreNLP.
Stanza proporciona una buena integración entre Python y CoreNLP.
Es importante recordar que cada opcion tiene sus ventajas y desventajas.
La opcion de ngrok, puede tener restricciones de tiempo y uso gratuito.

In [21]:
!pip install stanza
import stanza

In [34]:
import stanza

# --- 1. Instalar Stanza y descargar el modelo CoreNLP ---
stanza.download('en')

# --- 2. Configurar la canalización CoreNLP ---
# Inicializar la canalización CoreNLP
nlp = stanza.Pipeline('en', processors='tokenize,mwt,pos,lemma,ner')

text = """The Road from ‘Citizens United’ to Trump, Musk, and Corruption
Newsom's ties to CCP under microscope in new book exposing alleged corruption: 'Fleeced American citizens'
Tennis chair ump banned 6 years for corruption
Opinion | Here’s How New York Police Officers Start Their Day by Breaking the Law
What to Know About the Status of the Eric Adams Corruption Case
Lawyer appointed in NYC Mayor Eric Adams' case recommends dropping charges
Mayor Adams’ corruption charges should be dropped for good, court-appointed lawyer says
Feds seek San Leandro records naming Duong family as corruption probe expands beyond Oakland
Crack down on corruption and deploy tech – Xi’s twin tasks for the military
Sheng Thao corruption case: US attorney's office seeks Duong family records from city of San Leandro following Bryan Azevedo raid
Ex-Liverpool mayor Joe Anderson charged with bribery in corruption probe
‘Like a virus’: Corruption has infected the fight against climate change
If You Can Keep It: The Trump administration targets anti-corruption measures
A Simple Way to Check Police Corruption? Parking Tickets.
STOLEN FUTURES: The impact of corruption on children in Africa.
Murphy: Six Weeks In, This White House Is On Its Way To Being The Most Corrupt In U.S. History
How to help Nigerians who want to act against corruption
Blackfeet Nation takes action against corruption within legal team
Israeli prime minister appears in court for 16th time in his corruption trial
‘Mickey 17’ explores the human horrors of colonization, corruption and cloning
Ukraine Is Trying To Root Out Corruption. Is It Enough To Silence Critics?
AHA! Launches Anti-Corruption Task Force to Combat Food Fraud and Industry Deception
Corruption Perceptions Index 2024
Swiss court acquits Dahdaleh 12 years after UK corruption trial collapse
Iran Protests Erupt Over Housing, Pension Rights, and Government Corruption
Opinion | Making corruption great again
Drop Eric Adams Charges, Outside Counsel Advises Corruption Case Judge
Algeria: court orders detention of ex-interior minister for ‘corruption’
Letter | Vote Crawford to reject chaos and corruption
Whistleblowers in Office of Inspector General allege widespread corruption at highest levels of city government
Eric Adams investigation: Lawyer appointed as advisor recommends dismissing mayor's corruption case permanently
The ABCs of the CPI: How the Corruption Perceptions Index is calculated
Judge Dismisses Corruption Charges Against George Norcross, State Plans Appeal
Overcoming corruption vital for RI’s economy to ‘take off’, says Ray Dalio
Singapore working with Indonesia to extradite corruption fugitive: minister
Court-appointed lawyer tells judge to drop Eric Adams corruption case permanently
Have federal agents served warrants at California’s Capitol? The Legislature doesn’t want you to know
New York City Mayor Eric Adams' corruption case should be dismissed with prejudice, outside attorney tells judge
ByteDance dismisses hundreds of employees for corruption
Female students mark International Women's day by leading protests in Serbia against corruption
1015: Dial up corruption in Kurdistan! - Shafaq News
2 men arrested after reportedly trying to kill possible witness in Sheng Thao corruption case
Exclusive: Two men were arrested after attempted killing of potential witness in Sheng Thao corruption case
My Take | Here we go again: Ismail Sabri is third Malaysian ex-PM on corruption carousel
Why Serbian protesters set off smoke bombs in parliament
Corruption and waste, two dangers that threaten Brussels' plan to Rearm Europe
Nigeria's corruption watchdog recovers nearly $500 million in one year
Federal trial in Jackson corruption case set for summer 2026
Israeli protesters call Netanyahu 'corrupt' outside his trial
‘Profound betrayal’: 8 charged in North Charleston corruption investigation
Foreign journalists on China's anti-corruption drive: a global fight
Report: Despite Corruption Problems, China Progresses Toward Modernization
Trump’s 'Crypto Reserve' Is Such Brazen Corruption
BOMBSHELL VERDICT: Jury Confirms Corruption and Retaliation at the Highest Levels of Kansas City Leadership
Trump Weakens Anti-Corruption Measures, Potentially Hindering His Own Fight Against Fentanyl
Pertamina’s US$12 billion graft probe fuels Indonesians’ anger over ‘oil mafia’
Israeli PM Benjamin Netanyahu arrives in court to continue his testimony in ongoing corruption trial
4 of 8 charged in North Charleston corruption case plead guilty in federal court
Mayor's aides questioned by prosecution in corruption probe
Tennis chair umpire slapped with six-year ban after 'facilitating corruption'
Mapped: Which Countries Are Perceived as the Most Corrupt?
The Nantha Kumar Case Is Destroying The Integrity Of The Malaysian Anti-Corruption Commission – Analysis
Bardstown residents complain of latest corruption case after sheriff, chief deputy were indicted
<0xE2><0x80><0x8B>Deputy provincial governor jailed as corruption investigation ramps up<0xE2><0x80><0x8B>
Eric Adams’ corruption case should be permanently dismissed, court-appointed lawyer says
Dominican umpire suspended until 2030 for breaching anti-corruption code
Kosta Diamantis and Chris Ziogas charged in corruption scheme
2024 Corruption Perceptions Index: Corruption is playing a devastating role in the climate crisis
Strategic corruption reserve
Trump's Justice Department hits the brakes on anti-corruption enforcement
2024 Corruption Perceptions Index: Corruption fuels environmental crime across the Americas
Embattled Huntington Park mayor vows stalled pool project will proceed despite corruption probe
Israeli PM Netanyahu Returns To Court For Testimony In Corruption Trial, Defends Telecommunications Reforms
How corruption erodes support for democracy
Head of ARMA Duma is trying to regain trust with a new memorandum after the conflict with anti-corruption activists
Crypto, Crime & Corruption: A memecoin family’s checkered past puts the presidency of Argentina’s Javier M
When It Comes To Being an Officer, It’s Time To Go When Corruption Reigns
Israel's Benjamin Netanyahu takes stand in ongoing corruption trial
Workshop held to equip prosecutors combat corruption and money laundering
Somali Regional State denies corruption allegations, dismisses claims as “false” and part of “defamation campaign”
Denmark and Finland on top of list of least corrupt countries
Eric Adams' corruption case should be dismissed permanently, third-party attorney tells court
Xi calls on China’s military to tighten anti-corruption controls
Department of Prisons cracks down on corruption
Pellegrini suit alleging Hoboken corruption dismissed after his embezzlement plea
Are U.S. ‘news deserts’ hothouses of corruption?
Chaos and Corruption Weekly Digest: Week 5
Assistant Governor and accomplices arrested in $400,000 corruption bust
Israeli PM Benjamin Netanyahu arrives in court to continue his testimony in ongoing corruption trial
S/Court Decides Tweah’s, Others’ Fate in US$6.1M Corruption Case Today
Prisons dept cracks down on corruption
DeSoto County DA announces two corruption convictions
Trump and Musk Hunt for Corruption, Very Selectively
The International Anti-Corruption Day on 9 December 2024 is fast approaching!
Tahkout Corruption Scandal: New Trial of Property Ownership
Alabama city divided over whether to abolish police department accused of "culture of corruption"
HOS urges workers to support anti-corruption campaign
AHA! Launches Anti-Corruption Task Force to Combat Food Fraud and Industry Deception
Nigeria's corruption watchdog recovers nearly $500 million in one year
2025/2026 budget: Opposition to focus on corruption fight
Tennis chair umpire slapped with six-year ban after 'facilitating corruption'
Edo has regressed under Gov Okpebholo with massive corruption – Dr. Aziegbemi
ACU arrests Deputy Governor, two officials for corruption
LISTEN | Corruption, incompetence and a dark city: How City Power, Prasa flout the tender process
Tahkout Corruption Scandal: New Trial of Property Ownership
Swiss court acquits Dahdaleh 12 years after UK corruption trial collapse"""

# --- 4. Procesar el texto con CoreNLP ---
# Procesar el texto con CoreNLP
doc = nlp(text)

# --- 5. Acceder a los resultados ---
# Iterar sobre las oraciones y los tokens
for sent in doc.sentences:
    print(f"Oración: {sent.text}")
    for token in sent.tokens:
        print(f"  Token: {token.text}, POS: {token.words[0].upos}, Lema: {token.words[0].lemma}, NER: {token.ner}")

# También puedes acceder a las entidades nombradas
print("\nEntidades nombradas:")
for ent in doc.ents:
    print(f"  Entidad: {ent.text}, Tipo: {ent.type}")

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: en (English) ...
INFO:stanza:File exists: /root/stanza_resources/en/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: en (English):
| Processor | Package                   |
-----------------------------------------
| tokenize  | combined                  |
| mwt       | combined                  |
| pos       | combined_charlm           |
| lemma     | combined_nocharlm         |
| ner       | ontonotes-ww-multi_charlm |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


Oración: The Road from ‘Citizens United’ to Trump, Musk, and Corruption
Newsom's ties to CCP under microscope in new book exposing alleged corruption: 'Fleeced American citizens'
  Token: The, POS: DET, Lema: the, NER: O
  Token: Road, POS: PROPN, Lema: Road, NER: O
  Token: from, POS: ADP, Lema: from, NER: O
  Token: ‘, POS: PUNCT, Lema: ', NER: O
  Token: Citizens, POS: PROPN, Lema: citizen, NER: B-ORG
  Token: United, POS: ADJ, Lema: United, NER: E-ORG
  Token: ’, POS: PUNCT, Lema: ', NER: O
  Token: to, POS: ADP, Lema: to, NER: O
  Token: Trump, POS: PROPN, Lema: Trump, NER: S-PERSON
  Token: ,, POS: PUNCT, Lema: ,, NER: O
  Token: Musk, POS: PROPN, Lema: musk, NER: S-ORG
  Token: ,, POS: PUNCT, Lema: ,, NER: O
  Token: and, POS: CCONJ, Lema: and, NER: O
  Token: Corruption, POS: PROPN, Lema: Corruption, NER: B-ORG
  Token: Newsom's, POS: PROPN, Lema: Newsom, NER: E-ORG
  Token: ties, POS: NOUN, Lema: tie, NER: O
  Token: to, POS: ADP, Lema: to, NER: O
  Token: CCP, POS: PROPN, Lem

Reemplaza 'tu_archivo.txt': Asegúrate de reemplazarlo con la ruta correcta a tu archivo. Si el archivo no está en el directorio de trabajo de Colab, puedes subirlo usando el icono de la carpeta en la barra lateral izquierda.
Personalización: Puedes personalizar los procesadores en stanza.Pipeline() según tus necesidades.
Acceso a resultados: El código muestra cómo iterar sobre oraciones, tokens y entidades, y cómo acceder a información específica como lemas y personas.
Guardar resultados: Se incluye un ejemplo de cómo guardar los lemas en un archivo de texto.
Manejo de archivos grandes: Si el archivo es muy grande, considera procesarlo por partes para evitar problemas de memoria.
Idioma: Cambia 'en' en stanza.download() y stanza.Pipeline() para procesar otros idiomas.
Error de archivo no encontrado: Si aparece un error indicando que el archivo no existe, verifica que el archivo este subido a colab, y que el nombre del archivo en el codigo sea el mismo al nombre del archivo subido.

In [35]:
import stanza

# --- 1. Instalar Stanza y descargar el modelo CoreNLP ---
stanza.download('en')  # Descargar el modelo en inglés

# --- 2. Configurar la canalización CoreNLP ---
# Inicializar la canalización CoreNLP
nlp = stanza.Pipeline('en', processors='tokenize,mwt,pos,lemma,ner')

# --- 3. Texto de ejemplo en inglés ---
text = """The Road from ‘Citizens United’ to Trump, Musk, and Corruption
Newsom's ties to CCP under microscope in new book exposing alleged corruption: 'Fleeced American citizens'
Tennis chair ump banned 6 years for corruption
Opinion | Here’s How New York Police Officers Start Their Day by Breaking the Law
What to Know About the Status of the Eric Adams Corruption Case
Lawyer appointed in NYC Mayor Eric Adams' case recommends dropping charges
Mayor Adams’ corruption charges should be dropped for good, court-appointed lawyer says
Feds seek San Leandro records naming Duong family as corruption probe expands beyond Oakland
Crack down on corruption and deploy tech – Xi’s twin tasks for the military
Sheng Thao corruption case: US attorney's office seeks Duong family records from city of San Leandro following Bryan Azevedo raid
Ex-Liverpool mayor Joe Anderson charged with bribery in corruption probe
‘Like a virus’: Corruption has infected the fight against climate change
If You Can Keep It: The Trump administration targets anti-corruption measures
A Simple Way to Check Police Corruption? Parking Tickets.
STOLEN FUTURES: The impact of corruption on children in Africa.
Murphy: Six Weeks In, This White House Is On Its Way To Being The Most Corrupt In U.S. History
How to help Nigerians who want to act against corruption
Blackfeet Nation takes action against corruption within legal team
Israeli prime minister appears in court for 16th time in his corruption trial
‘Mickey 17’ explores the human horrors of colonization, corruption and cloning
Ukraine Is Trying To Root Out Corruption. Is It Enough To Silence Critics?
AHA! Launches Anti-Corruption Task Force to Combat Food Fraud and Industry Deception
Corruption Perceptions Index 2024
Swiss court acquits Dahdaleh 12 years after UK corruption trial collapse
Iran Protests Erupt Over Housing, Pension Rights, and Government Corruption
Opinion | Making corruption great again
Drop Eric Adams Charges, Outside Counsel Advises Corruption Case Judge
Algeria: court orders detention of ex-interior minister for ‘corruption’
Letter | Vote Crawford to reject chaos and corruption
Whistleblowers in Office of Inspector General allege widespread corruption at highest levels of city government
Eric Adams investigation: Lawyer appointed as advisor recommends dismissing mayor's corruption case permanently
The ABCs of the CPI: How the Corruption Perceptions Index is calculated
Judge Dismisses Corruption Charges Against George Norcross, State Plans Appeal
Overcoming corruption vital for RI’s economy to ‘take off’, says Ray Dalio
Singapore working with Indonesia to extradite corruption fugitive: minister
Court-appointed lawyer tells judge to drop Eric Adams corruption case permanently
Have federal agents served warrants at California’s Capitol? The Legislature doesn’t want you to know
New York City Mayor Eric Adams' corruption case should be dismissed with prejudice, outside attorney tells judge
ByteDance dismisses hundreds of employees for corruption
Female students mark International Women's day by leading protests in Serbia against corruption
1015: Dial up corruption in Kurdistan! - Shafaq News
2 men arrested after reportedly trying to kill possible witness in Sheng Thao corruption case
Exclusive: Two men were arrested after attempted killing of potential witness in Sheng Thao corruption case
My Take | Here we go again: Ismail Sabri is third Malaysian ex-PM on corruption carousel
Why Serbian protesters set off smoke bombs in parliament
Corruption and waste, two dangers that threaten Brussels' plan to Rearm Europe
Nigeria's corruption watchdog recovers nearly $500 million in one year
Federal trial in Jackson corruption case set for summer 2026
Israeli protesters call Netanyahu 'corrupt' outside his trial
‘Profound betrayal’: 8 charged in North Charleston corruption investigation
Foreign journalists on China's anti-corruption drive: a global fight
Report: Despite Corruption Problems, China Progresses Toward Modernization
Trump’s 'Crypto Reserve' Is Such Brazen Corruption
BOMBSHELL VERDICT: Jury Confirms Corruption and Retaliation at the Highest Levels of Kansas City Leadership
Trump Weakens Anti-Corruption Measures, Potentially Hindering His Own Fight Against Fentanyl
Pertamina’s US$12 billion graft probe fuels Indonesians’ anger over ‘oil mafia’
Israeli PM Benjamin Netanyahu arrives in court to continue his testimony in ongoing corruption trial
4 of 8 charged in North Charleston corruption case plead guilty in federal court
Mayor's aides questioned by prosecution in corruption probe
Tennis chair umpire slapped with six-year ban after 'facilitating corruption'
Mapped: Which Countries Are Perceived as the Most Corrupt?
The Nantha Kumar Case Is Destroying The Integrity Of The Malaysian Anti-Corruption Commission – Analysis
Bardstown residents complain of latest corruption case after sheriff, chief deputy were indicted
<0xE2><0x80><0x8B>Deputy provincial governor jailed as corruption investigation ramps up<0xE2><0x80><0x8B>
Eric Adams’ corruption case should be permanently dismissed, court-appointed lawyer says
Dominican umpire suspended until 2030 for breaching anti-corruption code
Kosta Diamantis and Chris Ziogas charged in corruption scheme
2024 Corruption Perceptions Index: Corruption is playing a devastating role in the climate crisis
Strategic corruption reserve
Trump's Justice Department hits the brakes on anti-corruption enforcement
2024 Corruption Perceptions Index: Corruption fuels environmental crime across the Americas
Embattled Huntington Park mayor vows stalled pool project will proceed despite corruption probe
Israeli PM Netanyahu Returns To Court For Testimony In Corruption Trial, Defends Telecommunications Reforms
How corruption erodes support for democracy
Head of ARMA Duma is trying to regain trust with a new memorandum after the conflict with anti-corruption activists
Crypto, Crime & Corruption: A memecoin family’s checkered past puts the presidency of Argentina’s Javier M
When It Comes To Being an Officer, It’s Time To Go When Corruption Reigns
Israel's Benjamin Netanyahu takes stand in ongoing corruption trial
Workshop held to equip prosecutors combat corruption and money laundering
Somali Regional State denies corruption allegations, dismisses claims as “false” and part of “defamation campaign”
Denmark and Finland on top of list of least corrupt countries
Eric Adams' corruption case should be dismissed permanently, third-party attorney tells court
Xi calls on China’s military to tighten anti-corruption controls
Department of Prisons cracks down on corruption
Pellegrini suit alleging Hoboken corruption dismissed after his embezzlement plea
Are U.S. ‘news deserts’ hothouses of corruption?
Chaos and Corruption Weekly Digest: Week 5
Assistant Governor and accomplices arrested in $400,000 corruption bust
Israeli PM Benjamin Netanyahu arrives in court to continue his testimony in ongoing corruption trial
S/Court Decides Tweah’s, Others’ Fate in US$6.1M Corruption Case Today
Prisons dept cracks down on corruption
DeSoto County DA announces two corruption convictions
Trump and Musk Hunt for Corruption, Very Selectively
The International Anti-Corruption Day on 9 December 2024 is fast approaching!
Tahkout Corruption Scandal: New Trial of Property Ownership
Alabama city divided over whether to abolish police department accused of "culture of corruption"
HOS urges workers to support anti-corruption campaign
AHA! Launches Anti-Corruption Task Force to Combat Food Fraud and Industry Deception
Nigeria's corruption watchdog recovers nearly $500 million in one year
2025/2026 budget: Opposition to focus on corruption fight
Tennis chair umpire slapped with six-year ban after 'facilitating corruption'
Edo has regressed under Gov Okpebholo with massive corruption – Dr. Aziegbemi
ACU arrests Deputy Governor, two officials for corruption
LISTEN | Corruption, incompetence and a dark city: How City Power, Prasa flout the tender process
Tahkout Corruption Scandal: New Trial of Property Ownership
Swiss court acquits Dahdaleh 12 years after UK corruption trial collapse"""
# --- 4. Procesar el texto con CoreNLP ---
# Procesar el texto con CoreNLP
doc = nlp(text)

# --- 5. Acceder a los resultados ---
# Iterar sobre las oraciones y los tokens
for sent in doc.sentences:
    print(f"Oración: {sent.text}")
    for token in sent.tokens:
        print(f"  Token: {token.text}, POS: {token.words[0].upos}, Lema: {token.words[0].lemma}, NER: {token.ner}")

# También puedes acceder a las entidades nombradas
print("\nEntidades nombradas:")
for ent in doc.ents:
    print(f"  Entidad: {ent.text}, Tipo: {ent.type}")

# --- 6. Ejemplo de acceso a información específica ---

# Ejemplo: Obtener todos los lemas
lemas = [word.lemma for sent in doc.sentences for token in sent.tokens for word in token.words]
print("\nLemas:", lemas)

# Ejemplo: Obtener todas las entidades nombradas de tipo PERSON
personas = [ent.text for ent in doc.ents if ent.type == "PERSON"]
print("\nPersonas:", personas)

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: en (English) ...
INFO:stanza:File exists: /root/stanza_resources/en/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: en (English):
| Processor | Package                   |
-----------------------------------------
| tokenize  | combined                  |
| mwt       | combined                  |
| pos       | combined_charlm           |
| lemma     | combined_nocharlm         |
| ner       | ontonotes-ww-multi_charlm |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


Oración: The Road from ‘Citizens United’ to Trump, Musk, and Corruption
Newsom's ties to CCP under microscope in new book exposing alleged corruption: 'Fleeced American citizens'
  Token: The, POS: DET, Lema: the, NER: O
  Token: Road, POS: PROPN, Lema: Road, NER: O
  Token: from, POS: ADP, Lema: from, NER: O
  Token: ‘, POS: PUNCT, Lema: ', NER: O
  Token: Citizens, POS: PROPN, Lema: citizen, NER: B-ORG
  Token: United, POS: ADJ, Lema: United, NER: E-ORG
  Token: ’, POS: PUNCT, Lema: ', NER: O
  Token: to, POS: ADP, Lema: to, NER: O
  Token: Trump, POS: PROPN, Lema: Trump, NER: S-PERSON
  Token: ,, POS: PUNCT, Lema: ,, NER: O
  Token: Musk, POS: PROPN, Lema: musk, NER: S-ORG
  Token: ,, POS: PUNCT, Lema: ,, NER: O
  Token: and, POS: CCONJ, Lema: and, NER: O
  Token: Corruption, POS: PROPN, Lema: Corruption, NER: B-ORG
  Token: Newsom's, POS: PROPN, Lema: Newsom, NER: E-ORG
  Token: ties, POS: NOUN, Lema: tie, NER: O
  Token: to, POS: ADP, Lema: to, NER: O
  Token: CCP, POS: PROPN, Lem

In [36]:
%%javascript
async function analyzeText(text) {
  const ngrokURL = "https://dashboard.ngrok.com/get-started/your-authtoken "; // Reemplaza con tu URL de ngrok
  const response = await fetch(ngrokURL + '/?properties={"annotators": "tokenize,ssplit,pos,lemma,ner,parse,sentiment", "outputFormat": "json"}', {
    method: 'POST',
    body: text
  });
  const data = await response.json();
  return data;
}

const textToAnalyze = "This is a positive sentence. This is a negative one.";

analyzeText(textToAnalyze).then(result => {
  console.log(result);
});

<IPython.core.display.Javascript object>

In [37]:
!cd stanford-corenlp-full-2020-04-20 && java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 15000 &

/bin/bash: line 1: cd: stanford-corenlp-full-2020-04-20: No such file or directory


In [38]:
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
!tar -xzf ngrok-v3-stable-linux-amd64.tgz
!./ngrok authtoken 2h1F1Ydnhk6SGILXYIHlPKiZn9p_3qJmVbuQVtxYvjSBssDU6 # Reemplaza TU_AUTHTOKEN
!./ngrok http 9000 &
!curl -s http://localhost:4040/api/tunnels

--2025-03-17 13:19:28--  https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 75.2.60.68, 99.83.220.108, 13.248.244.96, ...
Connecting to bin.equinox.io (bin.equinox.io)|75.2.60.68|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9371732 (8.9M) [application/octet-stream]
Saving to: ‘ngrok-v3-stable-linux-amd64.tgz.1’

ngrok-v3-stable-lin 100%[===================>]   8.94M  --.-KB/s    in 0.1s    

2025-03-17 13:19:28 (91.4 MB/s) - ‘ngrok-v3-stable-linux-amd64.tgz.1’ saved [9371732/9371732]

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


Importante: Reemplaza TU_URL_NGROK con la URL pública que obtuviste de ngrok.
Explicación:

El código de shell inicia el servidor CoreNLP y lo expone a través de ngrok.
El código JavaScript define una función analyzeText que:
Toma un texto como entrada.
Envía una solicitud POST al servidor CoreNLP a través de la URL de ngrok.
Solicita análisis semántico completo (tokenización, separación de oraciones, POS tagging, lematización, NER, análisis sintáctico y análisis de sentimiento).
Solicita la salida en formato JSON.
Devuelve los resultados del análisis como un objeto JSON.
El código JavaScript luego llama a analyzeText con un texto de ejemplo y muestra los resultados en la consola del navegador.
Pasos para ejecutar:

Ejecuta las celdas de código de shell para iniciar el servidor CoreNLP y ngrok.
Copia la URL pública de ngrok de la salida de !curl -s http://localhost:4040/api/tunnels.
Pega la URL de ngrok en el código JavaScript, reemplazando TU_URL_NGROK.
Reemplaza TU_AUTHTOKEN con tu authtoken de ngrok.
Ejecuta la celda de código JavaScript.
Abre la consola del navegador (clic derecho > Inspeccionar > Consola) para ver la salida JSON.
Consideraciones:

Asegúrate de que el servidor CoreNLP esté funcionando antes de ejecutar el código JavaScript.
La URL de ngrok cambiará cada vez que reinicies el túnel, así que tendrás que actualizarla en el código JavaScript.
El analisis de sentimiento solo funciona correctamente en ingles.
Recuerda que ngrok tiene algunas limitaciones en su versión gratuita, como la duración de los túneles.

In [40]:
!cd stanford-corenlp-full-2020-04-20 && java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 15000 &

/bin/bash: line 1: cd: stanford-corenlp-full-2020-04-20: No such file or directory


In [41]:
# Descargar CoreNLP (si aún no lo has hecho)
!wget http://nlp.stanford.edu/software/stanford-corenlp-full-2020-04-20.zip
!unzip stanford-corenlp-full-2020-04-20.zip

# Iniciar el servidor CoreNLP
!cd stanford-corenlp-full-2020-04-20 && java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 15000 &

--2025-03-17 13:24:56--  http://nlp.stanford.edu/software/stanford-corenlp-full-2020-04-20.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/software/stanford-corenlp-full-2020-04-20.zip [following]
--2025-03-17 13:24:56--  https://nlp.stanford.edu/software/stanford-corenlp-full-2020-04-20.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://downloads.cs.stanford.edu/nlp/software/stanford-corenlp-full-2020-04-20.zip [following]
--2025-03-17 13:24:56--  https://downloads.cs.stanford.edu/nlp/software/stanford-corenlp-full-2020-04-20.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... conn

In [43]:
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
!tar -xzf ngrok-v3-stable-linux-amd64.tgz
!./ngrok authtoken 2h1F1Ydnhk6SGILXYIHlPKiZn9p_3qJmVbuQVtxYvjSBssDU6 # Reemplaza TU_AUTHTOKEN
!./ngrok http 9000 &
!curl -s http://localhost:4040/api/tunnelsy


--2025-03-17 13:29:32--  https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 75.2.60.68, 99.83.220.108, 13.248.244.96, ...
Connecting to bin.equinox.io (bin.equinox.io)|75.2.60.68|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9371732 (8.9M) [application/octet-stream]
Saving to: ‘ngrok-v3-stable-linux-amd64.tgz.3’

ngrok-v3-stable-lin 100%[===================>]   8.94M  --.-KB/s    in 0.06s   

2025-03-17 13:29:32 (138 MB/s) - ‘ngrok-v3-stable-linux-amd64.tgz.3’ saved [9371732/9371732]

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [45]:
%%javascript
async function analyzeText(text) {
  const ngrokURL = "https://dashboard.ngrok.com/get-started/your-authtoken"; // Reemplaza TU_URL_NGROK
  const response = await fetch(ngrokURL + '/?properties={"annotators": "tokenize,ssplit,pos,lemma,ner,parse,sentiment", "outputFormat": "json"}', {
    method: 'POST',
    body: text
  });
  const data = await response.json();
  console.log(JSON.stringify(data, null, 2)); // Imprimir JSON formateado en la consola
  return data;
}

const textToAnalyze = "Este es un ejemplo de texto para analizar";


analyzeText(textToAnalyze);

<IPython.core.display.Javascript object>